# DDINTER SYNTOX ANALYSIS #
File to analyze relationship between synergy scores and toxicity levels

In [1]:
# Import everything needed
from matplotlib.patches import Patch
from scipy import stats
from sklearn.metrics import r2_score
from statsmodels.stats.multitest import multipletests
from preprocessing_functions import *
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scikit_posthocs as sp

In [2]:
# Use drug_syntox dataframe that was created by preprocessing all databases
drug_syntox_df = pd.read_csv("data_processed/ddinter_processed_combos_syntoxtargallpw_string.csv")
print('DDInter fully processed dataframe shape: ', drug_syntox_df.shape)
num_entries_major = drug_syntox_df['toxicity_category'].value_counts()['Major']
num_entries_moderate = drug_syntox_df['toxicity_category'].value_counts()['Moderate']
num_entries_minor = drug_syntox_df['toxicity_category'].value_counts()['Minor']
print('Number of Major entries: ', num_entries_major)
print('Number of Moderate entries: ', num_entries_moderate)
print('Number of Minor entries: ', num_entries_minor)
num_unique_drugs = set(drug_syntox_df['drug_row'].unique()).union(set(drug_syntox_df['drug_col'].unique()))
print('Number of unique drugs: ', len(num_unique_drugs))
unique_major_pairs = set()
unique_moderate_pairs = set()
unique_minor_pairs = set()
for index, row in drug_syntox_df.iterrows():
    first_way = (row['drug_row'], row['drug_col'])
    second_way = (row['drug_col'], row['drug_row'])
    if row['toxicity_category'] == 'Major':
        if first_way not in unique_major_pairs and second_way not in unique_major_pairs:
            unique_major_pairs.add(first_way)
    elif row['toxicity_category'] == 'Moderate':
        if first_way not in unique_moderate_pairs and second_way not in unique_moderate_pairs:
            unique_moderate_pairs.add(first_way)
    elif row['toxicity_category'] == 'Minor':
        if first_way not in unique_minor_pairs and second_way not in unique_minor_pairs:
            unique_minor_pairs.add(first_way)
print('Number of unique Major pairs: ', len(unique_major_pairs))
print('Number of unique Moderate pairs: ', len(unique_moderate_pairs))
print('Number of unique Minor pairs: ', len(unique_minor_pairs))


DDInter fully processed dataframe shape:  (29064, 12)
Number of Major entries:  5776
Number of Moderate entries:  22457
Number of Minor entries:  831
Number of unique drugs:  331
Number of unique Major pairs:  679
Number of unique Moderate pairs:  1920
Number of unique Minor pairs:  116


In [3]:
# Create histogram for each synergy score: bliss, loewe, hsa, zip, s_max, s_mean, s_sum
drug_syntox_df.hist(column='synergy_bliss', bins=100)
plt.xlabel('Bliss synergy score')
plt.ylabel('Frequency')
plt.title('Bliss synergy score distribution in DrugComb-DDInter intersection')
plt.savefig('results/synergy_score_distrib/bliss_hist_ddinter.png')
plt.close()

drug_syntox_df.hist(column='synergy_loewe', bins=100)
plt.xlabel('Loewe synergy score')
plt.ylabel('Frequency')
plt.title('Loewe synergy score distribution in DrugComb-DDInter intersection')
plt.savefig('results/synergy_score_distrib/loewe_hist_ddinter.png')
plt.close()

drug_syntox_df.hist(column='synergy_hsa', bins=100)
plt.xlabel('HSA synergy score')
plt.ylabel('Frequency')
plt.title('HSA synergy score distribution in DrugComb-DDInter intersection')
plt.savefig('results/synergy_score_distrib/hsa_hist_ddinter.png')
plt.close()

drug_syntox_df.hist(column='synergy_zip', bins=100)
plt.xlabel('ZIP synergy score')
plt.ylabel('Frequency')
plt.title('ZIP synergy score distribution in DrugComb-DDInter intersection')
plt.savefig('results/synergy_score_distrib/zip_hist_ddinter.png')
plt.close()

drug_syntox_df.hist(column='S_max', bins=100)
plt.xlabel('S_max')
plt.ylabel('Frequency')
plt.title('S_max distribution in DrugComb-DDInter intersection')
plt.savefig('results/synergy_score_distrib/smax_hist_ddinter.png')
plt.close()

drug_syntox_df.hist(column='S_mean', bins=100)
plt.xlabel('S_mean')
plt.ylabel('Frequency')
plt.title('S_mean distribution in DrugComb-DDInter intersection')
plt.savefig('results/synergy_score_distrib/smean_hist_ddinter.png')
plt.close()

drug_syntox_df.hist(column='S_sum', bins=100)
plt.xlabel('S_sum')
plt.ylabel('Frequency')
plt.title('S_sum distribution in DrugComb-DDInter intersection')
plt.savefig('results/synergy_score_distrib/ssum_hist_ddinter.png')
plt.close()

# Write the max, min, and normality test to four significant digits to a file, if less than 0.05, then not normal
with open('results/synergy_score_distrib/synergy_stats_ddinter.txt', 'w') as f:
    f.write('Max Bliss synergy: {:.4e}\n'.format(drug_syntox_df['synergy_bliss'].max()))
    f.write('Min Bliss synergy: {:.4e}\n'.format(drug_syntox_df['synergy_bliss'].min()))
    f.write('Bliss synergy normality test: {:.4e}\n'.format(stats.normaltest(drug_syntox_df['synergy_bliss'])[1]))
    f.write('Max HSA synergy: {:.4e}\n'.format(drug_syntox_df['synergy_hsa'].max()))
    f.write('Min HSA synergy: {:.4e}\n'.format(drug_syntox_df['synergy_hsa'].min()))
    f.write('HSA synergy normality test: {:.4e}\n'.format(stats.normaltest(drug_syntox_df['synergy_hsa'])[1]))
    f.write('Max Loewe synergy: {:.4e}\n'.format(drug_syntox_df['synergy_loewe'].max()))
    f.write('Min Loewe synergy: {:.4e}\n'.format(drug_syntox_df['synergy_loewe'].min()))
    f.write('Loewe synergy normality test: {:.4e}\n'.format(stats.normaltest(drug_syntox_df['synergy_loewe'])[1]))
    f.write('Max ZIP synergy: {:.4e}\n'.format(drug_syntox_df['synergy_zip'].max()))
    f.write('Min ZIP synergy: {:.4e}\n'.format(drug_syntox_df['synergy_zip'].min()))
    f.write('ZIP synergy normality test: {:.4e}\n'.format(stats.normaltest(drug_syntox_df['synergy_zip'])[1]))
    f.write('Max S_max: {:.4e}\n'.format(drug_syntox_df['S_max'].max()))
    f.write('Min S_max: {:.4e}\n'.format(drug_syntox_df['S_max'].min()))
    f.write('S_max normality test: {:.4e}\n'.format(stats.normaltest(drug_syntox_df['S_max'])[1]))
    f.write('Max S_mean: {:.4e}\n'.format(drug_syntox_df['S_mean'].max()))
    f.write('Min S_mean: {:.4e}\n'.format(drug_syntox_df['S_mean'].min()))
    f.write('S_mean normality test: {:.4e}\n'.format(stats.normaltest(drug_syntox_df['S_mean'])[1]))
    f.write('Max S_sum: {:.4e}\n'.format(drug_syntox_df['S_sum'].max()))
    f.write('Min S_sum: {:.4e}\n'.format(drug_syntox_df['S_sum'].min()))
    f.write('S_sum normality test: {:.4e}\n'.format(stats.normaltest(drug_syntox_df['S_sum'])[1]))

In [4]:
############ BLISS ##############
# Create histogram for synergy_bliss for only Major toxicity category in drug_syntox_df
drug_syntox_major = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Major']
drug_syntox_major['synergy_bliss'].hist(bins=100)
plt.xlabel('Synergy Bliss')
plt.ylabel('Frequency')
plt.title('Synergy Bliss Distribution for Major Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/bliss_major_hist_ddinter.png')
plt.close()

# Create histogram for synergy_bliss for only Moderate toxicity category in drug_syntox_df
drug_syntox_moderate = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate']
drug_syntox_moderate['synergy_bliss'].hist(bins=100)
plt.xlabel('Synergy Bliss')
plt.ylabel('Frequency')
plt.title('Synergy Bliss Distribution for Moderate Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/bliss_moderate_hist_ddinter.png')
plt.close()

# Create histogram for synergy_bliss for only Minor toxicity category in drug_syntox_df
drug_syntox_minor = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Minor']
drug_syntox_minor['synergy_bliss'].hist(bins=100)
plt.xlabel('Synergy Bliss')
plt.ylabel('Frequency')
plt.title('Synergy Bliss Distribution for Minor Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/bliss_minor_hist_ddinter.png')
plt.close()

############ HSA ##############
# Create histogram for synergy_HSA for only Major toxicity category in drug_syntox_df
drug_syntox_major['synergy_hsa'].hist(bins=100)
plt.xlabel('Synergy HSA')
plt.ylabel('Frequency')
plt.title('Synergy HSA Distribution for Major Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/hsa_major_hist_ddinter.png')
plt.close()

# Create histogram for synergy_HSA for only Moderate toxicity category in drug_syntox_df
drug_syntox_moderate['synergy_hsa'].hist(bins=100)
plt.xlabel('Synergy HSA')
plt.ylabel('Frequency')
plt.title('Synergy HSA Distribution for Moderate Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/hsa_moderate_hist_ddinter.png')
plt.close()

# Create histogram for synergy_HSA for only Minor toxicity category in drug_syntox_df
drug_syntox_minor['synergy_hsa'].hist(bins=100)
plt.xlabel('Synergy HSA')
plt.ylabel('Frequency')
plt.title('Synergy HSA Distribution for Minor Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/hsa_minor_hist_ddinter.png')
plt.close()

############ LOEWE ##############
# Create histogram for synergy_loewe for only Major toxicity category in drug_syntox_df
drug_syntox_major['synergy_loewe'].hist(bins=100)
plt.xlabel('Synergy Loewe')
plt.ylabel('Frequency')
plt.title('Synergy Loewe Distribution for Major Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/loewe_major_hist_ddinter.png')
plt.close()

# Create histogram for synergy_loewe for only Moderate toxicity category in drug_syntox_df
drug_syntox_moderate['synergy_loewe'].hist(bins=100)
plt.xlabel('Synergy Loewe')
plt.ylabel('Frequency')
plt.title('Synergy Loewe Distribution for Moderate Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/loewe_moderate_hist_ddinter.png')
plt.close()

# Create histogram for synergy_loewe for only Minor toxicity category in drug_syntox_df
drug_syntox_minor['synergy_loewe'].hist(bins=100)
plt.xlabel('Synergy Loewe')
plt.ylabel('Frequency')
plt.title('Synergy Loewe Distribution for Minor Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/loewe_minor_hist_ddinter.png')
plt.close()

############ ZIP ##############
# Create histogram for synergy_zip for only Major toxicity category in drug_syntox_df
drug_syntox_major['synergy_zip'].hist(bins=100)
plt.xlabel('Synergy ZIP')
plt.ylabel('Frequency')
plt.title('Synergy ZIP Distribution for Major Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/zip_major_hist_ddinter.png')
plt.close()

# Create histogram for synergy_zip for only Moderate toxicity category in drug_syntox_df
drug_syntox_moderate['synergy_zip'].hist(bins=100)
plt.xlabel('Synergy ZIP')
plt.ylabel('Frequency')
plt.title('Synergy ZIP Distribution for Moderate Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/zip_moderate_hist_ddinter.png')
plt.close()

# Create histogram for synergy_zip for only Minor toxicity category in drug_syntox_df
drug_syntox_minor['synergy_zip'].hist(bins=100)
plt.xlabel('Synergy ZIP')
plt.ylabel('Frequency')
plt.title('Synergy ZIP Distribution for Minor Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/zip_minor_hist_ddinter.png')
plt.close()

############ S_MAX ##############
# Create histogram for S_max for only Major toxicity category in drug_syntox_df
drug_syntox_major['S_max'].hist(bins=100)
plt.xlabel('S_max')
plt.ylabel('Frequency')
plt.title('S_max Distribution for Major Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/smax_major_hist_ddinter.png')
plt.close()

# Create histogram for S_max for only Moderate toxicity category in drug_syntox_df
drug_syntox_moderate['S_max'].hist(bins=100)
plt.xlabel('S_max')
plt.ylabel('Frequency')
plt.title('S_max Distribution for Moderate Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/smax_moderate_hist_ddinter.png')
plt.close()

# Create histogram for S_max for only Minor toxicity category in drug_syntox_df
drug_syntox_minor['S_max'].hist(bins=100)
plt.xlabel('S_max')
plt.ylabel('Frequency')
plt.title('S_max Distribution for Minor Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/smax_minor_hist_ddinter.png')
plt.close()

############ S_MEAN ##############
# Create histogram for S_mean for only Major toxicity category in drug_syntox_df
drug_syntox_major['S_mean'].hist(bins=100)
plt.xlabel('S_mean')
plt.ylabel('Frequency')
plt.title('S_mean Distribution for Major Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/smean_major_hist_ddinter.png')
plt.close()

# Create histogram for S_mean for only Moderate toxicity category in drug_syntox_df
drug_syntox_moderate['S_mean'].hist(bins=100)
plt.xlabel('S_mean')
plt.ylabel('Frequency')
plt.title('S_mean Distribution for Moderate Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/smean_moderate_hist_ddinter.png')
plt.close()

# Create histogram for S_mean for only Minor toxicity category in drug_syntox_df
drug_syntox_minor['S_mean'].hist(bins=100)
plt.xlabel('S_mean')
plt.ylabel('Frequency')
plt.title('S_mean Distribution for Minor Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/smean_minor_hist_ddinter.png')
plt.close()

############ S_SUM ##############
# Create histogram for S_sum for only Major toxicity category in drug_syntox_df
drug_syntox_major['S_sum'].hist(bins=100)
plt.xlabel('S_sum')
plt.ylabel('Frequency')
plt.title('S_sum Distribution for Major Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/ssum_major_hist_ddinter.png')
plt.close()

# Create histogram for S_sum for only Moderate toxicity category in drug_syntox_df
drug_syntox_moderate['S_sum'].hist(bins=100)
plt.xlabel('S_sum')
plt.ylabel('Frequency')
plt.title('S_sum Distribution for Moderate Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/ssum_moderate_hist_ddinter.png')
plt.close()

# Create histogram for S_sum for only Minor toxicity category in drug_syntox_df
drug_syntox_minor['S_sum'].hist(bins=100)
plt.xlabel('S_sum')
plt.ylabel('Frequency')
plt.title('S_sum Distribution for Minor Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/ssum_minor_hist_ddinter.png')
plt.close()

# Write Mean, Median, Variance, and Standard Deviation to 4 significant figures to a file
with open('results/synergy_score_distrib/ddinter_summary_stats.csv', 'w') as f:
    f.write('Synergy Score, Toxicity Category, Mean, Standard Deviation, Median, Variance\n')
    f.write('Bliss, All, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_df['synergy_bliss'].mean(),
        drug_syntox_df['synergy_bliss'].std(),
        drug_syntox_df['synergy_bliss'].median(),
        drug_syntox_df['synergy_bliss'].var()))
    f.write('Bliss, Major, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_major['synergy_bliss'].mean(),
        drug_syntox_major['synergy_bliss'].std(),
        drug_syntox_major['synergy_bliss'].median(),
        drug_syntox_major['synergy_bliss'].var()))
    f.write('Bliss, Moderate, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_moderate['synergy_bliss'].mean(),
        drug_syntox_moderate['synergy_bliss'].std(),
        drug_syntox_moderate['synergy_bliss'].median(),
        drug_syntox_moderate['synergy_bliss'].var()))
    f.write('Bliss, Minor, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_minor['synergy_bliss'].mean(),
        drug_syntox_minor['synergy_bliss'].std(),
        drug_syntox_minor['synergy_bliss'].median(),
        drug_syntox_minor['synergy_bliss'].var()))
    f.write('HSA, All, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_df['synergy_hsa'].mean(),
        drug_syntox_df['synergy_hsa'].std(),
        drug_syntox_df['synergy_hsa'].median(),
        drug_syntox_df['synergy_hsa'].var()))
    f.write('HSA, Major, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_major['synergy_hsa'].mean(),
        drug_syntox_major['synergy_hsa'].std(),
        drug_syntox_major['synergy_hsa'].median(),
        drug_syntox_major['synergy_hsa'].var()))
    f.write('HSA, Moderate, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_moderate['synergy_hsa'].mean(),
        drug_syntox_moderate['synergy_hsa'].std(),
        drug_syntox_moderate['synergy_hsa'].median(),
        drug_syntox_moderate['synergy_hsa'].var()))
    f.write('HSA, Minor, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_minor['synergy_hsa'].mean(),
        drug_syntox_minor['synergy_hsa'].std(),
        drug_syntox_minor['synergy_hsa'].median(),
        drug_syntox_minor['synergy_hsa'].var()))
    f.write('Loewe, All, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_df['synergy_loewe'].mean(),
        drug_syntox_df['synergy_loewe'].std(),
        drug_syntox_df['synergy_loewe'].median(),
        drug_syntox_df['synergy_loewe'].var()))
    f.write('Loewe, Major, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_major['synergy_loewe'].mean(),
        drug_syntox_major['synergy_loewe'].std(),
        drug_syntox_major['synergy_loewe'].median(),
        drug_syntox_major['synergy_loewe'].var()))
    f.write('Loewe, Moderate, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_moderate['synergy_loewe'].mean(),
        drug_syntox_moderate['synergy_loewe'].std(),
        drug_syntox_moderate['synergy_loewe'].median(),
        drug_syntox_moderate['synergy_loewe'].var()))
    f.write('Loewe, Minor, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_minor['synergy_loewe'].mean(),
        drug_syntox_minor['synergy_loewe'].std(),
        drug_syntox_minor['synergy_loewe'].median(),
        drug_syntox_minor['synergy_loewe'].var()))
    f.write('ZIP, All, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_df['synergy_zip'].mean(),
        drug_syntox_df['synergy_zip'].std(),
        drug_syntox_df['synergy_zip'].median(),
        drug_syntox_df['synergy_zip'].var()))
    f.write('ZIP, Major, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_major['synergy_zip'].mean(),
        drug_syntox_major['synergy_zip'].std(),
        drug_syntox_major['synergy_zip'].median(),
        drug_syntox_major['synergy_zip'].var()))
    f.write('ZIP, Moderate, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_moderate['synergy_zip'].mean(),
        drug_syntox_moderate['synergy_zip'].std(),
        drug_syntox_moderate['synergy_zip'].median(),
        drug_syntox_moderate['synergy_zip'].var()))
    f.write('ZIP, Minor, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_minor['synergy_zip'].mean(),
        drug_syntox_minor['synergy_zip'].std(),
        drug_syntox_minor['synergy_zip'].median(),
        drug_syntox_minor['synergy_zip'].var()))
    f.write('S_max, All, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_df['S_max'].mean(),
        drug_syntox_df['S_max'].std(),
        drug_syntox_df['S_max'].median(),
        drug_syntox_df['S_max'].var()))
    f.write('S_max, Major, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_major['S_max'].mean(),
        drug_syntox_major['S_max'].std(),
        drug_syntox_major['S_max'].median(),
        drug_syntox_major['S_max'].var()))
    f.write('S_max, Moderate, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_moderate['S_max'].mean(),
        drug_syntox_moderate['S_max'].std(),
        drug_syntox_moderate['S_max'].median(),
        drug_syntox_moderate['S_max'].var()))
    f.write('S_max, Minor, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_minor['S_max'].mean(),
        drug_syntox_minor['S_max'].std(),
        drug_syntox_minor['S_max'].median(),
        drug_syntox_minor['S_max'].var()))
    f.write('S_mean, All, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_df['S_mean'].mean(),
        drug_syntox_df['S_mean'].std(),
        drug_syntox_df['S_mean'].median(),
        drug_syntox_df['S_mean'].var()))
    f.write('S_mean, Major, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_major['S_mean'].mean(),
        drug_syntox_major['S_mean'].std(),
        drug_syntox_major['S_mean'].median(),
        drug_syntox_major['S_mean'].var()))
    f.write('S_mean, Moderate, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_moderate['S_mean'].mean(),
        drug_syntox_moderate['S_mean'].std(),
        drug_syntox_moderate['S_mean'].median(),
        drug_syntox_moderate['S_mean'].var()))
    f.write('S_mean, Minor, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_minor['S_mean'].mean(),
        drug_syntox_minor['S_mean'].std(),
        drug_syntox_minor['S_mean'].median(),
        drug_syntox_minor['S_mean'].var()))
    f.write('S_sum, All, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_df['S_sum'].mean(),
        drug_syntox_df['S_sum'].std(),
        drug_syntox_df['S_sum'].median(),
        drug_syntox_df['S_sum'].var()))
    f.write('S_sum, Major, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_major['S_sum'].mean(),
        drug_syntox_major['S_sum'].std(),
        drug_syntox_major['S_sum'].median(),
        drug_syntox_major['S_sum'].var()))
    f.write('S_sum, Moderate, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_moderate['S_sum'].mean(),
        drug_syntox_moderate['S_sum'].std(),
        drug_syntox_moderate['S_sum'].median(),
        drug_syntox_moderate['S_sum'].var()))
    f.write('S_sum, Minor, {:.4e}, {:.4e}, {:.4e}, {:.4e}\n'.format(
        drug_syntox_minor['S_sum'].mean(),
        drug_syntox_minor['S_sum'].std(),
        drug_syntox_minor['S_sum'].median(),
        drug_syntox_minor['S_sum'].var()))
    
    

- Kruskal Wallis Test
- Dunn Posthoc Test with Bonferroni Correction
- Jonckheere Terpestra Test
- ANOVA
- T Test with Bonferroni Correction

In [5]:
# Are there significant differences in the distributions of synergy scores between Major, Moderate, and Minor toxicity categories?
# Filter drug_syntox_df for only Major, Moderate, and Minor toxicity categories
drug_syntox_df = drug_syntox_df[drug_syntox_df['toxicity_category'].isin(['Major', 'Moderate', 'Minor'])]
colors = ['#20965D', '#FFBC42', '#D81159']
tox_order = ['Minor', 'Moderate', 'Major']
color_dict = dict(zip(tox_order, colors))
legend_elements = [
    Patch(facecolor=color_dict[cat], label = cat) for cat in tox_order
]

#Bliss
major_bliss = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Major']['synergy_bliss']
moderate_bliss = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate']['synergy_bliss']
minor_bliss = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Minor']['synergy_bliss']

h_statistic_bliss, p_value_bliss = stats.kruskal(major_bliss, moderate_bliss, minor_bliss)
print(f'Kruskal-Wallis H statistic for Bliss synergy: {h_statistic_bliss}')
print(f'P-value for Bliss synergy: {p_value_bliss}')

dunn_bliss = sp.posthoc_dunn(drug_syntox_df, val_col='synergy_bliss', group_col='toxicity_category', p_adjust='bonferroni')
print('Dunn post-hoc test for Bliss synergy:')
print(dunn_bliss)

bliss_major_samples = major_bliss.values.tolist()
bliss_moderate_samples = moderate_bliss.values.tolist()
bliss_minor_samples = minor_bliss.values.tolist()
bliss_increasing_tox_samples = [bliss_minor_samples, bliss_moderate_samples, bliss_major_samples]
jt_incr_bliss = jonckheere_terpestra_test(bliss_increasing_tox_samples)
print("Increasing toxicity for Bliss: ", jt_incr_bliss )
bliss_decreasing_tox_samples = [ bliss_major_samples, bliss_moderate_samples, bliss_minor_samples]
jt_decr_bliss = jonckheere_terpestra_test(bliss_decreasing_tox_samples)
print("Decreasing toxicity for Bliss: ", jt_decr_bliss)

# Run an ANOVA on the Bliss synergy scores for Major, Moderate, and Minor toxicity categoreis
f_statistic_anova_bliss_tox, p_value_anova_bliss_tox = stats.f_oneway(major_bliss, moderate_bliss, minor_bliss)
print(f'ANOVA F-statistic for bliss: {f_statistic_anova_bliss_tox}')
print(f'P-value for bliss: {p_value_anova_bliss_tox}')

# Run a T test on bliss between Major/Minor, Major/Moderate, and Moderate/Minor toxicity categories
major_minor_bliss = stats.ttest_ind(major_bliss, minor_bliss)
major_moderate_bliss = stats.ttest_ind(major_bliss, moderate_bliss)
moderate_minor_bliss = stats.ttest_ind(moderate_bliss, minor_bliss)

# Run bonferroni correction on the p-values
ttest_p_values_bliss = [major_minor_bliss[1], major_moderate_bliss[1], moderate_minor_bliss[1]]
ttest_p_values_corrected_bliss = multipletests(ttest_p_values_bliss, method='bonferroni')
print(f'Major/Minor T-test: {major_minor_bliss[1]} Corrected: {ttest_p_values_corrected_bliss[1][0]}')
print(f'Major/Moderate T-test: {major_moderate_bliss[1]} Corrected: {ttest_p_values_corrected_bliss[1][1]}')
print(f'Moderate/Minor T-test: {moderate_minor_bliss[1]} Corrected: {ttest_p_values_corrected_bliss[1][2]}')

ax = sns.violinplot(data=drug_syntox_df, x='toxicity_category', y='synergy_bliss', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set(xlabel='', ylabel='Bliss Synergy Score')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/figure2/bliss_v_toxcats_ddinter_violin.png', dpi=700)
plt.tight_layout()
plt.close()

ax = sns.stripplot(data=drug_syntox_df, x='toxicity_category', y='synergy_bliss', palette=color_dict, hue='toxicity_category', order=tox_order)
sns.boxplot( # plot the mean line
    showmeans=True,
    meanline=True,
    meanprops={'color': 'k', 'ls': '-', 'lw': 1},
    medianprops={'visible': False},
    whiskerprops={'visible': False},
    zorder=10,
    x="toxicity_category",
    y="synergy_bliss",
    data=drug_syntox_df,
    showfliers=False,
    showbox=False,
    showcaps=False,
    ax=ax
)
ax.set(xlabel='', ylabel='Bliss Synergy Score')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/figure2/bliss_v_toxcats_ddinter_strip.png', dpi=700)
plt.tight_layout()
plt.close()

#HSA
major_hsa = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Major']['synergy_hsa']
moderate_hsa = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate']['synergy_hsa']
minor_hsa = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Minor']['synergy_hsa']

h_statistic_hsa, p_value_hsa = stats.kruskal(major_hsa, moderate_hsa, minor_hsa)
print(f'Kruskal-Wallis H statistic for HSA synergy: {h_statistic_hsa}')
print(f'P-value for HSA synergy: {p_value_hsa}')

dunn_hsa = sp.posthoc_dunn(drug_syntox_df, val_col='synergy_hsa', group_col='toxicity_category', p_adjust='bonferroni')
print('Dunn post-hoc test for HSA synergy:')
print(dunn_hsa)

hsa_major_samples = major_hsa.values.tolist()
hsa_moderate_samples = moderate_hsa.values.tolist()
hsa_minor_samples = minor_hsa.values.tolist()
hsa_increasing_tox_samples = [hsa_minor_samples, hsa_moderate_samples, hsa_major_samples]
jt_incr_hsa = jonckheere_terpestra_test(hsa_increasing_tox_samples)
print("Increasing toxicity for HSA: ", jt_incr_hsa )
hsa_decreasing_tox_samples = [ hsa_major_samples, hsa_moderate_samples, hsa_minor_samples]
jt_decr_hsa = jonckheere_terpestra_test(hsa_decreasing_tox_samples)
print("Decreasing toxicity for HSA: ", jt_decr_hsa)

# Run an ANOVA on the HSA synergy scores for Major, Moderate, and Minor toxicity categoreis
f_statistic_anova_hsa_tox, p_value_anova_hsa_tox = stats.f_oneway(major_hsa, moderate_hsa, minor_hsa)
print(f'ANOVA F-statistic for HSA: {f_statistic_anova_hsa_tox}')
print(f'P-value for HSA: {p_value_anova_hsa_tox}')

# Run a T test on HSA between Major/Minor, Major/Moderate, and Moderate/Minor toxicity categories
major_minor_hsa = stats.ttest_ind(major_hsa, minor_hsa)
major_moderate_hsa = stats.ttest_ind(major_hsa, moderate_hsa)
moderate_minor_hsa = stats.ttest_ind(moderate_hsa, minor_hsa)

# Run bonferroni correction on the p-values
ttest_p_values_hsa = [major_minor_hsa[1], major_moderate_hsa[1], moderate_minor_hsa[1]]
ttest_p_values_corrected_hsa = multipletests(ttest_p_values_hsa, method='bonferroni')
print(f'Major/Minor T-test: {major_minor_hsa[1]} Corrected: {ttest_p_values_corrected_hsa[1][0]}')
print(f'Major/Moderate T-test: {major_moderate_hsa[1]} Corrected: {ttest_p_values_corrected_hsa[1][1]}')
print(f'Moderate/Minor T-test: {moderate_minor_hsa[1]} Corrected: {ttest_p_values_corrected_hsa[1][2]}')

ax = sns.violinplot(data=drug_syntox_df, x='toxicity_category', y='synergy_hsa', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set(xlabel='', ylabel='HSA Synergy Score')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/figure2/hsa_v_toxcats_ddinter_violin.png', dpi=700)
plt.close()

ax = sns.stripplot(data=drug_syntox_df, x='toxicity_category', y='synergy_hsa', palette=color_dict, hue='toxicity_category', order=tox_order)
sns.boxplot( # plot the mean line
    showmeans=True,
    meanline=True,
    meanprops={'color': 'k', 'ls': '-', 'lw': 1},
    medianprops={'visible': False},
    whiskerprops={'visible': False},
    zorder=10,
    x="toxicity_category",
    y="synergy_hsa",
    data=drug_syntox_df,
    showfliers=False,
    showbox=False,
    showcaps=False,
    ax=ax
)
ax.set(xlabel='', ylabel='HSA Synergy Score')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/figure2/hsa_v_toxcats_ddinter_strip.png', dpi=700)
plt.tight_layout()
plt.close()

#ZIP
major_zip = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Major']['synergy_zip']
moderate_zip = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate']['synergy_zip']
minor_zip = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Minor']['synergy_zip']

h_statistic_zip, p_value_zip = stats.kruskal(major_zip, moderate_zip, minor_zip)
print(f'Kruskal-Wallis H statistic for ZIP synergy: {h_statistic_zip}')
print(f'P-value for ZIP synergy: {p_value_zip}')

dunn_zip = sp.posthoc_dunn(drug_syntox_df, val_col='synergy_zip', group_col='toxicity_category', p_adjust='bonferroni')
print('Dunn post-hoc test for ZIP synergy:')
print(dunn_zip)

zip_major_samples = major_zip.values.tolist()
zip_moderate_samples = moderate_zip.values.tolist()
zip_minor_samples = minor_zip.values.tolist()
zip_increasing_tox_samples = [zip_minor_samples, zip_moderate_samples, zip_major_samples]
jt_incr_zip = jonckheere_terpestra_test(zip_increasing_tox_samples)
print("Increasing toxicity for ZIP: ", jt_incr_zip )
zip_decreasing_tox_samples = [ zip_major_samples, zip_moderate_samples, zip_minor_samples]
jt_decr_zip = jonckheere_terpestra_test(zip_decreasing_tox_samples)
print("Decreasing toxicity for ZIP: ", jt_decr_zip)

# Run an ANOVA on the ZIP synergy scores for Major, Moderate, and Minor toxicity categoreis
f_statistic_anova_zip_tox, p_value_anova_zip_tox = stats.f_oneway(major_zip, moderate_zip, minor_zip)
print(f'ANOVA F-statistic for ZIP: {f_statistic_anova_zip_tox}')
print(f'P-value for ZIP: {p_value_anova_zip_tox}')

# Run a T test on ZIP between Major/Minor, Major/Moderate, and Moderate/Minor toxicity categories
major_minor_zip = stats.ttest_ind(major_zip, minor_zip)
major_moderate_zip = stats.ttest_ind(major_zip, moderate_zip)
moderate_minor_zip = stats.ttest_ind(moderate_zip, minor_zip)

# Run bonferroni correction on the p-values
ttest_p_values_zip = [major_minor_zip[1], major_moderate_zip[1], moderate_minor_zip[1]]
ttest_p_values_corrected_zip = multipletests(ttest_p_values_zip, method='bonferroni')
print(f'Major/Minor T-test: {major_minor_zip[1]} Corrected: {ttest_p_values_corrected_zip[1][0]}')
print(f'Major/Moderate T-test: {major_moderate_zip[1]} Corrected: {ttest_p_values_corrected_zip[1][1]}')
print(f'Moderate/Minor T-test: {moderate_minor_zip[1]} Corrected: {ttest_p_values_corrected_zip[1][2]}')

ax = sns.violinplot(data=drug_syntox_df, x='toxicity_category', y='synergy_zip', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set(xlabel='', ylabel='ZIP Synergy Score')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/figure2/zip_v_toxcats_ddinter_violin.png', dpi=700)
plt.close()

ax = sns.stripplot(data=drug_syntox_df, x='toxicity_category', y='synergy_zip', palette=color_dict, hue='toxicity_category', order=tox_order)
sns.boxplot( # plot the mean line
    showmeans=True,
    meanline=True,
    meanprops={'color': 'k', 'ls': '-', 'lw': 1},
    medianprops={'visible': False},
    whiskerprops={'visible': False},
    zorder=10,
    x="toxicity_category",
    y="synergy_zip",
    data=drug_syntox_df,
    showfliers=False,
    showbox=False,
    showcaps=False,
    ax=ax
)
ax.set(xlabel='', ylabel='ZIP Synergy Score')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/figure2/zip_v_toxcats_ddinter_strip.png', dpi=700)
plt.close()

#Loewe
major_loewe = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Major']['synergy_loewe']
moderate_loewe = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate']['synergy_loewe']
minor_loewe = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Minor']['synergy_loewe']

h_statistic_loewe, p_value_loewe = stats.kruskal(major_loewe, moderate_loewe, minor_loewe)
print(f'Kruskal-Wallis H statistic for Loewe synergy: {h_statistic_loewe}')
print(f'P-value for Loewe synergy: {p_value_loewe}')

dunn_loewe = sp.posthoc_dunn(drug_syntox_df, val_col='synergy_loewe', group_col='toxicity_category', p_adjust='bonferroni')
print('Dunn post-hoc test for Loewe synergy:')
print(dunn_loewe)

loewe_major_samples = major_loewe.values.tolist()
loewe_moderate_samples = moderate_loewe.values.tolist()
loewe_minor_samples = minor_loewe.values.tolist()
loewe_increasing_tox_samples = [loewe_minor_samples, loewe_moderate_samples, loewe_major_samples]
jt_incr_loewe = jonckheere_terpestra_test(loewe_increasing_tox_samples)
print("Increasing toxicity for Loewe: ", jt_incr_loewe )
loewe_decreasing_tox_samples = [ loewe_major_samples, loewe_moderate_samples, loewe_minor_samples]
jt_decr_loewe = jonckheere_terpestra_test(loewe_decreasing_tox_samples)
print("Decreasing toxicity for Loewe: ", jt_decr_loewe)

# Run an ANOVA on the Loewe synergy scores for Major, Moderate, and Minor toxicity categoreis
f_statistic_anova_loewe_tox, p_value_anova_loewe_tox = stats.f_oneway(major_loewe, moderate_loewe, minor_loewe)
print(f'ANOVA F-statistic for Loewe: {f_statistic_anova_loewe_tox}')
print(f'P-value for Loewe: {p_value_anova_loewe_tox}')

# Run a T test on Loewe between Major/Minor, Major/Moderate, and Moderate/Minor toxicity categories
major_minor_loewe = stats.ttest_ind(major_loewe, minor_loewe)
major_moderate_loewe = stats.ttest_ind(major_loewe, moderate_loewe)
moderate_minor_loewe = stats.ttest_ind(moderate_loewe, minor_loewe)

# Run bonferroni correction on the p-values
ttest_p_values_loewe = [major_minor_loewe[1], major_moderate_loewe[1], moderate_minor_loewe[1]]
ttest_p_values_corrected_loewe = multipletests(ttest_p_values_loewe, method='bonferroni')
print(f'Major/Minor T-test: {major_minor_loewe[1]} Corrected: {ttest_p_values_corrected_loewe[1][0]}')
print(f'Major/Moderate T-test: {major_moderate_loewe[1]} Corrected: {ttest_p_values_corrected_loewe[1][1]}')
print(f'Moderate/Minor T-test: {moderate_minor_loewe[1]} Corrected: {ttest_p_values_corrected_loewe[1][2]}')

ax = sns.violinplot(data=drug_syntox_df, x='toxicity_category', y='synergy_loewe', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set(xlabel='', ylabel='Loewe Synergy Score')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/figure2/loewe_v_toxcats_ddinter_violin.png', dpi=700)
plt.close()

ax = sns.stripplot(data=drug_syntox_df, x='toxicity_category', y='synergy_loewe', palette=color_dict, hue='toxicity_category', order=tox_order)
sns.boxplot( # plot the mean line
    showmeans=True,
    meanline=True,
    meanprops={'color': 'k', 'ls': '-', 'lw': 1},
    medianprops={'visible': False},
    whiskerprops={'visible': False},
    zorder=10,
    x="toxicity_category",
    y="synergy_loewe",
    data=drug_syntox_df,
    showfliers=False,
    showbox=False,
    showcaps=False,
    ax=ax
)
ax.set(xlabel='', ylabel='Loewe Synergy Score')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/figure2/loewe_v_toxcats_ddinter_strip.png', dpi=700)
plt.close()

#S_max
major_smax = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Major']['S_max']
moderate_smax = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate']['S_max']
minor_smax = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Minor']['S_max']

h_statistic_smax, p_value_smax = stats.kruskal(major_smax, moderate_smax, minor_smax)
print(f'Kruskal-Wallis H statistic for S_max: {h_statistic_smax}')
print(f'P-value for S_max: {p_value_smax}')
dunn_smax = sp.posthoc_dunn(drug_syntox_df, val_col='S_max', group_col='toxicity_category', p_adjust='bonferroni')
print('Dunn post-hoc test for S_max:')
print(dunn_smax)

smax_major_samples = major_smax.values.tolist()
smax_moderate_samples = moderate_smax.values.tolist()
smax_minor_samples = minor_smax.values.tolist()
smax_increasing_tox_samples = [smax_minor_samples, smax_moderate_samples, smax_major_samples]
jt_incr_smax = jonckheere_terpestra_test(smax_increasing_tox_samples)
print("Increasing toxicity for S_max: ", jt_incr_smax )
smax_decreasing_tox_samples = [ smax_major_samples, smax_moderate_samples, smax_minor_samples]
jt_decr_smax = jonckheere_terpestra_test(smax_decreasing_tox_samples)
print("Decreasing toxicity for S_max: ", jt_decr_smax)

# Run an ANOVA on the S_max scores for Major, Moderate, and Minor toxicity categoreis
f_statistic_anova_smax_tox, p_value_anova_smax_tox = stats.f_oneway(major_smax, moderate_smax, minor_smax)
print(f'ANOVA F-statistic for S_max: {f_statistic_anova_smax_tox}')
print(f'P-value for S_max: {p_value_anova_smax_tox}')

# Run a T test on S_max between Major/Minor, Major/Moderate, and Moderate/Minor toxicity categories
major_minor_smax = stats.ttest_ind(major_smax, minor_smax)
major_moderate_smax = stats.ttest_ind(major_smax, moderate_smax)
moderate_minor_smax = stats.ttest_ind(moderate_smax, minor_smax)

# Run bonferroni correction on the p-values
ttest_p_values_smax = [major_minor_smax[1], major_moderate_smax[1], moderate_minor_smax[1]]
ttest_p_values_corrected_smax = multipletests(ttest_p_values_smax, method='bonferroni')
print(f'Major/Minor T-test: {major_minor_smax[1]} Corrected: {ttest_p_values_corrected_smax[1][0]}')
print(f'Major/Moderate T-test: {major_moderate_smax[1]} Corrected: {ttest_p_values_corrected_smax[1][1]}')
print(f'Moderate/Minor T-test: {moderate_minor_smax[1]} Corrected: {ttest_p_values_corrected_smax[1][2]}')

ax = sns.violinplot(data=drug_syntox_df, x='toxicity_category', y='S_max', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set(xlabel='', ylabel='S_max Synergy Score')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/figure2/smax_v_toxcats_ddinter_violin.png', dpi=700)
plt.close()

ax = sns.stripplot(data=drug_syntox_df, x='toxicity_category', y='S_max', palette=color_dict, hue='toxicity_category', order=tox_order)
sns.boxplot( # plot the mean line
    showmeans=True,
    meanline=True,
    meanprops={'color': 'k', 'ls': '-', 'lw': 1},
    medianprops={'visible': False},
    whiskerprops={'visible': False},
    zorder=10,
    x="toxicity_category",
    y="S_max",
    data=drug_syntox_df,
    showfliers=False,
    showbox=False,
    showcaps=False,
    ax=ax
)
ax.set(xlabel='', ylabel='S_max Synergy Score')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/figure2/smax_v_toxcats_ddinter_strip.png', dpi=700)
plt.close()

#S_mean
major_smean = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Major']['S_mean']
moderate_smean = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate']['S_mean']
minor_smean = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Minor']['S_mean']
h_statistic_smean, p_value_smean = stats.kruskal(major_smean, moderate_smean, minor_smean)
print(f'Kruskal-Wallis H statistic for S_mean: {h_statistic_smean}')
print(f'P-value for S_mean: {p_value_smean}')
dunn_smean = sp.posthoc_dunn(drug_syntox_df, val_col='S_mean', group_col='toxicity_category', p_adjust='bonferroni')
print('Dunn post-hoc test for S_mean:')
print(dunn_smean)

smean_major_samples = major_smean.values.tolist()
smean_moderate_samples = moderate_smean.values.tolist()
smean_minor_samples = minor_smean.values.tolist()
smean_increasing_tox_samples = [smean_minor_samples, smean_moderate_samples, smean_major_samples]
jt_incr_smean = jonckheere_terpestra_test(smean_increasing_tox_samples)
print("Increasing toxicity for S_mean: ", jt_incr_smean )
smean_decreasing_tox_samples = [ smean_major_samples, smean_moderate_samples, smean_minor_samples]
jt_decr_smean = jonckheere_terpestra_test(smean_decreasing_tox_samples)
print("Decreasing toxicity for S_mean: ", jt_decr_smean)

# Run an ANOVA on the S_mean scores for Major, Moderate, and Minor toxicity categoreis
f_statistic_anova_smean_tox, p_value_anova_smean_tox = stats.f_oneway(major_smean, moderate_smean, minor_smean)
print(f'ANOVA F-statistic for S_mean: {f_statistic_anova_smean_tox}')
print(f'P-value for S_mean: {p_value_anova_smean_tox}')

# Run a T test on S_mean between Major/Minor, Major/Moderate, and Moderate/Minor toxicity categories
major_minor_smean = stats.ttest_ind(major_smean, minor_smean)
major_moderate_smean = stats.ttest_ind(major_smean, moderate_smean)
moderate_minor_smean = stats.ttest_ind(moderate_smean, minor_smean)

# Run bonferroni correction on the p-values
ttest_p_values_smean = [major_minor_smean[1], major_moderate_smean[1], moderate_minor_smean[1]]
ttest_p_values_corrected_smean = multipletests(ttest_p_values_smean, method='bonferroni')
print(f'Major/Minor T-test: {major_minor_smean[1]} Corrected: {ttest_p_values_corrected_smean[1][0]}')
print(f'Major/Moderate T-test: {major_moderate_smean[1]} Corrected: {ttest_p_values_corrected_smean[1][1]}')
print(f'Moderate/Minor T-test: {moderate_minor_smean[1]} Corrected: {ttest_p_values_corrected_smean[1][2]}')

ax = sns.violinplot(data=drug_syntox_df, x='toxicity_category', y='S_mean', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set(xlabel='', ylabel='S_mean Synergy Score')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/figure2/smean_v_toxcats_ddinter_violin.png', dpi=700)
plt.close()

ax = sns.stripplot(data=drug_syntox_df, x='toxicity_category', y='S_mean', palette=color_dict, hue='toxicity_category', order=tox_order)
sns.boxplot( # plot the mean line
    showmeans=True,
    meanline=True,
    meanprops={'color': 'k', 'ls': '-', 'lw': 1},
    medianprops={'visible': False},
    whiskerprops={'visible': False},
    zorder=10,
    x="toxicity_category",
    y="S_mean",
    data=drug_syntox_df,
    showfliers=False,
    showbox=False,
    showcaps=False,
    ax=ax
)
ax.set(xlabel='', ylabel='S_mean Synergy Score')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/figure2/smean_v_toxcats_ddinter_strip.png', dpi=700)
plt.close()

#S_sum
major_ssum = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Major']['S_sum']
moderate_ssum = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate']['S_sum']
minor_ssum = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Minor']['S_sum']
h_statistic_ssum, p_value_ssum = stats.kruskal(major_ssum, moderate_ssum, minor_ssum)
print(f'Kruskal-Wallis H statistic for S_sum: {h_statistic_ssum}')
print(f'P-value for S_sum: {p_value_ssum}')
dunn_ssum = sp.posthoc_dunn(drug_syntox_df, val_col='S_sum', group_col='toxicity_category', p_adjust='bonferroni')
print('Dunn post-hoc test for S_sum:')
print(dunn_ssum)

ssum_major_samples = major_ssum.values.tolist()
ssum_moderate_samples = moderate_ssum.values.tolist()
ssum_minor_samples = minor_ssum.values.tolist()
ssum_increasing_tox_samples = [ssum_minor_samples, ssum_moderate_samples, ssum_major_samples]
jt_incr_ssum = jonckheere_terpestra_test(ssum_increasing_tox_samples)
print("Increasing toxicity for S_sum: ", jt_incr_ssum )
ssum_decreasing_tox_samples = [ ssum_major_samples, ssum_moderate_samples, ssum_minor_samples]
jt_decr_ssum = jonckheere_terpestra_test(ssum_decreasing_tox_samples)
print("Decreasing toxicity for S_sum: ", jt_decr_ssum)

# Run an ANOVA on the S_sum scores for Major, Moderate, and Minor toxicity categoreis
f_statistic_anova_ssum_tox, p_value_ssum_tox = stats.f_oneway(major_ssum, moderate_ssum, minor_ssum)
print(f'ANOVA F-statistic for S_sum: {f_statistic_anova_ssum_tox}')
print(f'P-value for S_sum: {p_value_ssum_tox}')

# Run a T test on S_sum between Major/Minor, Major/Moderate, and Moderate/Minor toxicity categories
major_minor_ssum = stats.ttest_ind(major_ssum, minor_ssum)
major_moderate_ssum = stats.ttest_ind(major_ssum, moderate_ssum)
moderate_minor_ssum = stats.ttest_ind(moderate_ssum, minor_ssum)
# Run bonferroni correction on the p-values
ttest_p_values_ssum = [major_minor_ssum[1], major_moderate_ssum[1], moderate_minor_ssum[1]]
ttest_p_values_corrected_ssum = multipletests(ttest_p_values_ssum, method='bonferroni')
print(f'Major/Minor T-test: {major_minor_ssum[1]} Corrected: {ttest_p_values_corrected_ssum[1][0]}')
print(f'Major/Moderate T-test: {major_moderate_ssum[1]} Corrected: {ttest_p_values_corrected_ssum[1][1]}')
print(f'Moderate/Minor T-test: {moderate_minor_ssum[1]} Corrected: {ttest_p_values_corrected_ssum[1][2]}')

ax = sns.violinplot(data=drug_syntox_df, x='toxicity_category', y='S_sum', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set(xlabel='', ylabel='S_sum Synergy Score')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/figure2/ssum_v_toxcats_ddinter_violin.png', dpi=700)
plt.close()

ax = sns.stripplot(data=drug_syntox_df, x='toxicity_category', y='S_sum', palette=color_dict, hue='toxicity_category', order=tox_order)
sns.boxplot( # plot the mean line
    showmeans=True,
    meanline=True,
    meanprops={'color': 'k', 'ls': '-', 'lw': 1},
    medianprops={'visible': False},
    whiskerprops={'visible': False},
    zorder=10,
    x="toxicity_category",
    y="S_sum",
    data=drug_syntox_df,
    showfliers=False,
    showbox=False,
    showcaps=False,
    ax=ax
)
ax.set(xlabel='', ylabel='S_sum Synergy Score')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/figure2/ssum_v_toxcats_ddinter_strip.png', dpi=700)
plt.close()


# Write results to a file
with open('results/synergy_score_tox_categ_distrib/kruskal_dunn_jonck_majmodmin_syntox_ddinter.tsv', 'w') as f:
    f.write('Synergy Score\tTest\tTest Statistic\tP-value\n')
    f.write('Bliss\tKruskal-Wallis\t{:.4e}\t{:.4e}\n'.format(h_statistic_bliss, p_value_bliss))
    f.write('Bliss\tDunn post-hoc\tMajor/Minor\t{:.4e}\n'.format(dunn_bliss.iloc[0, 1]))
    f.write('Bliss\tDunn post-hoc\tMajor/Moderate\t{:.4e}\n'.format(dunn_bliss.iloc[0, 2]))
    f.write('Bliss\tDunn post-hoc\tModerate/Minor\t{:.4e}\n'.format(dunn_bliss.iloc[1, 2]))
    f.write('Bliss\tJonckheere-Terpstra Increasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_incr_bliss[0], jt_incr_bliss[1]))
    f.write('Bliss\tJonckheere-Terpstra Decreasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_decr_bliss[0], jt_decr_bliss[1]))
    f.write('Bliss\tANOVA\t{:.4e}\t{:.4e}\n'.format(f_statistic_anova_bliss_tox, p_value_anova_bliss_tox))
    f.write('Bliss\tT-test (bonf cor)\tMajor/Minor\t{:.4e}\n'.format(ttest_p_values_corrected_bliss[1][0]))
    f.write('Bliss\tT-test (bonf cor)\tMajor/Moderate\t{:.4e}\n'.format(ttest_p_values_corrected_bliss[1][1]))
    f.write('Bliss\tT-test (bonf cor)\tModerate/Minor\t{:.4e}\n'.format(ttest_p_values_corrected_bliss[1][2]))

    f.write('HSA\tKruskal-Wallis\t{:.4e}\t{:.4e}\n'.format(h_statistic_hsa, p_value_hsa))
    f.write('HSA\tDunn post-hoc\tMajor/Minor\t{:.4e}\n'.format(dunn_hsa.iloc[0, 1]))
    f.write('HSA\tDunn post-hoc\tMajor/Moderate\t{:.4e}\n'.format(dunn_hsa.iloc[0, 2]))
    f.write('HSA\tDunn post-hoc\tModerate/Minor\t{:.4e}\n'.format(dunn_hsa.iloc[1, 2]))
    f.write('HSA\tJonckheere-Terpstra Increasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_incr_hsa[0], jt_incr_hsa[1]))
    f.write('HSA\tJonckheere-Terpstra Decreasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_decr_hsa[0], jt_decr_hsa[1]))
    f.write('HSA\tANOVA\t{:.4e}\t{:.4e}\n'.format(f_statistic_anova_hsa_tox, p_value_anova_hsa_tox))
    f.write('HSA\tT-test (bonf cor)\tMajor/Minor\t{:.4e}\n'.format(ttest_p_values_corrected_hsa[1][0]))
    f.write('HSA\tT-test (bonf cor)\tMajor/Moderate\t{:.4e}\n'.format(ttest_p_values_corrected_hsa[1][1]))
    f.write('HSA\tT-test (bonf cor)\tModerate/Minor\t{:.4e}\n'.format(ttest_p_values_corrected_hsa[1][2]))

    f.write('Loewe\tKruskal-Wallis\t{:.4e}\t{:.4e}\n'.format(h_statistic_loewe, p_value_loewe))
    f.write('Loewe\tDunn post-hoc\tMajor/Minor\t{:.4e}\n'.format(dunn_loewe.iloc[0, 1]))
    f.write('Loewe\tDunn post-hoc\tMajor/Moderate\t{:.4e}\n'.format(dunn_loewe.iloc[0, 2]))
    f.write('Loewe\tDunn post-hoc\tModerate/Minor\t{:.4e}\n'.format(dunn_loewe.iloc[1, 2]))
    f.write('Loewe\tJonckheere-Terpstra Increasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_incr_loewe[0], jt_incr_loewe[1]))
    f.write('Loewe\tJonckheere-Terpstra Decreasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_decr_loewe[0], jt_decr_loewe[1]))
    f.write('Loewe\tANOVA\t{:.4e}\t{:.4e}\n'.format(f_statistic_anova_loewe_tox, p_value_anova_loewe_tox))
    f.write('Loewe\tT-test (bonf cor)\tMajor/Minor\t{:.4e}\n'.format(ttest_p_values_corrected_loewe[1][0]))
    f.write('Loewe\tT-test (bonf cor)\tMajor/Moderate\t{:.4e}\n'.format(ttest_p_values_corrected_loewe[1][1]))
    f.write('Loewe\tT-test (bonf cor)\tModerate/Minor\t{:.4e}\n'.format(ttest_p_values_corrected_loewe[1][2]))
    
    f.write('ZIP\tKruskal-Wallis\t{:.4e}\t{:.4e}\n'.format(h_statistic_zip, p_value_zip))
    f.write('ZIP\tDunn post-hoc\tMajor/Minor\t{:.4e}\n'.format(dunn_zip.iloc[0, 1]))
    f.write('ZIP\tDunn post-hoc\tMajor/Moderate\t{:.4e}\n'.format(dunn_zip.iloc[0, 2]))
    f.write('ZIP\tDunn post-hoc\tModerate/Minor\t{:.4e}\n'.format(dunn_zip.iloc[1, 2]))
    f.write('ZIP\tJonckheere-Terpstra Increasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_incr_zip[0], jt_incr_zip[1]))
    f.write('ZIP\tJonckheere-Terpstra Decreasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_decr_zip[0], jt_decr_zip[1]))
    f.write('ZIP\tANOVA\t{:.4e}\t{:.4e}\n'.format(f_statistic_anova_zip_tox, p_value_anova_zip_tox))
    f.write('ZIP\tT-test (bonf cor)\tMajor/Minor\t{:.4e}\n'.format(ttest_p_values_corrected_zip[1][0]))
    f.write('ZIP\tT-test (bonf cor)\tMajor/Moderate\t{:.4e}\n'.format(ttest_p_values_corrected_zip[1][1]))
    f.write('ZIP\tT-test (bonf cor)\tModerate/Minor\t{:.4e}\n'.format(ttest_p_values_corrected_zip[1][2]))
    
    f.write('S_max\tKruskal-Wallis\t{:.4e}\t{:.4e}\n'.format(h_statistic_smax, p_value_smax))
    f.write('S_max\tDunn post-hoc\tMajor/Minor\t{:.4e}\n'.format(dunn_smax.iloc[0, 1]))
    f.write('S_max\tDunn post-hoc\tMajor/Moderate\t{:.4e}\n'.format(dunn_smax.iloc[0, 2]))
    f.write('S_max\tDunn post-hoc\tModerate/Minor\t{:.4e}\n'.format(dunn_smax.iloc[1, 2]))
    f.write('S_max\tJonckheere-Terpstra Increasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_incr_smax[0], jt_incr_smax[1]))
    f.write('S_max\tJonckheere-Terpstra Decreasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_decr_smax[0], jt_decr_smax[1]))
    f.write('S_max\tANOVA\t{:.4e}\t{:.4e}\n'.format(f_statistic_anova_smax_tox, p_value_anova_smax_tox))
    f.write('S_max\tT-test (bonf cor)\tMajor/Minor\t{:.4e}\n'.format(ttest_p_values_corrected_smax[1][0]))
    f.write('S_max\tT-test (bonf cor)\tMajor/Moderate\t{:.4e}\n'.format(ttest_p_values_corrected_smax[1][1]))
    f.write('S_max\tT-test (bonf cor)\tModerate/Minor\t{:.4e}\n'.format(ttest_p_values_corrected_smax[1][2]))
    
    f.write('S_mean\tKruskal-Wallis\t{:.4e}\t{:.4e}\n'.format(h_statistic_smean, p_value_smean))
    f.write('S_mean\tDunn post-hoc\tMajor/Minor\t{:.4e}\n'.format(dunn_smean.iloc[0, 1]))
    f.write('S_mean\tDunn post-hoc\tMajor/Moderate\t{:.4e}\n'.format(dunn_smean.iloc[0, 2]))
    f.write('S_mean\tDunn post-hoc\tModerate/Minor\t{:.4e}\n'.format(dunn_smean.iloc[1, 2]))
    f.write('S_mean\tJonckheere-Terpstra Increasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_incr_smean[0], jt_incr_smean[1]))
    f.write('S_mean\tJonckheere-Terpstra Decreasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_decr_smean[0], jt_decr_smean[1]))
    f.write('S_mean\tANOVA\t{:.4e}\t{:.4e}\n'.format(f_statistic_anova_smean_tox, p_value_anova_smean_tox))
    f.write('S_mean\tT-test (bonf cor)\tMajor/Minor\t{:.4e}\n'.format(ttest_p_values_corrected_smean[1][0]))
    f.write('S_mean\tT-test (bonf cor)\tMajor/Moderate\t{:.4e}\n'.format(ttest_p_values_corrected_smean[1][1]))
    f.write('S_mean\tT-test (bonf cor)\tModerate/Minor\t{:.4e}\n'.format(ttest_p_values_corrected_smean[1][2]))
    
    f.write('S_sum\tKruskal-Wallis\t{:.4e}\t{:.4e}\n'.format(h_statistic_ssum, p_value_ssum))
    f.write('S_sum\tDunn post-hoc\tMajor/Minor\t{:.4e}\n'.format(dunn_ssum.iloc[0, 1]))
    f.write('S_sum\tDunn post-hoc\tMajor/Moderate\t{:.4e}\n'.format(dunn_ssum.iloc[0, 2]))
    f.write('S_sum\tDunn post-hoc\tModerate/Minor\t{:.4e}\n'.format(dunn_ssum.iloc[1, 2]))
    f.write('S_sum\tJonckheere-Terpstra Increasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_incr_ssum[0], jt_incr_ssum[1]))
    f.write('S_sum\tJonckheere-Terpstra Decreasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_decr_ssum[0], jt_decr_ssum[1]))
    f.write('S_sum\tANOVA\t{:.4e}\t{:.4e}\n'.format(f_statistic_anova_ssum_tox, p_value_ssum_tox))
    f.write('S_sum\tT-test (bonf cor)\tMajor/Minor\t{:.4e}\n'.format(ttest_p_values_corrected_ssum[1][0]))
    f.write('S_sum\tT-test (bonf cor)\tMajor/Moderate\t{:.4e}\n'.format(ttest_p_values_corrected_ssum[1][1]))
    f.write('S_sum\tT-test (bonf cor)\tModerate/Minor\t{:.4e}\n'.format(ttest_p_values_corrected_ssum[1][2]))


Kruskal-Wallis H statistic for Bliss synergy: 4.491392346948236
P-value for Bliss synergy: 0.10585382209309184
Dunn post-hoc test for Bliss synergy:
             Major     Minor  Moderate
Major     1.000000  0.108875  0.990113
Minor     0.108875  1.000000  0.219358
Moderate  0.990113  0.219358  1.000000
Increasing toxicity for Bliss:  (1.6444497933192124, 0.0500416634750277)
Decreasing toxicity for Bliss:  (-1.6444497933192124, 0.9499583365249723)
ANOVA F-statistic for bliss: 0.23459134927728337
P-value for bliss: 0.7908954807130613
Major/Minor T-test: 0.5478961116275101 Corrected: 1.0
Major/Moderate T-test: 0.7206489682688426 Corrected: 1.0
Moderate/Minor T-test: 0.5716526405219984 Corrected: 1.0
Kruskal-Wallis H statistic for HSA synergy: 519.8819671876954
P-value for HSA synergy: 1.2854796465255017e-113
Dunn post-hoc test for HSA synergy:
                 Major         Minor      Moderate
Major     1.000000e+00  2.127599e-02  4.678827e-94
Minor     2.127599e-02  1.000000e+00  7.6845

Plot the relationship between synergy scores, how do they relate to one another?

In [6]:
# Plot scatter plot of synergy_bliss vs. synergy_hsa
drug_syntox_df.plot.scatter(x='synergy_bliss', y='synergy_hsa')
plt.xlabel('Synergy Bliss', fontsize=20)
plt.ylabel('Synergy HSA', fontsize=20)
plt.title('Synergy HSA vs. Synergy Bliss', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/hsa_v_bliss_ddinter.png')
plt.close()

# Plot scatter plot of synergy_bliss vs. synergy_loewe
drug_syntox_df.plot.scatter(x='synergy_bliss', y='synergy_loewe')
plt.xlabel('Synergy Bliss', fontsize=20)
plt.ylabel('Synergy Loewe', fontsize=20)
plt.title('Synergy Loewe vs. Synergy Bliss', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/loewe_v_bliss_ddinter.png')
plt.close()

# Plot scatter plot of synergy_bliss vs. synergy_zip
drug_syntox_df.plot.scatter(x='synergy_bliss', y='synergy_zip')
plt.xlabel('Synergy Bliss', fontsize=20)
plt.ylabel('Synergy ZIP', fontsize=20)
plt.title('Synergy ZIP vs. Synergy Bliss', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/zip_v_bliss_ddinter.png')
plt.close()

# Plot scatter plot of synergy_bliss vs. S_max
drug_syntox_df.plot.scatter(x='synergy_bliss', y='S_max')
plt.xlabel('Synergy Bliss', fontsize=20)
plt.ylabel('S_max', fontsize=20)
plt.title('S_max vs. Synergy Bliss', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/smax_v_bliss_ddinter.png')
plt.close()

# Plot scatter plot of synergy_bliss vs. S_mean
drug_syntox_df.plot.scatter(x='synergy_bliss', y='S_mean')
plt.xlabel('Synergy Bliss', fontsize=20)
plt.ylabel('S_mean', fontsize=20)
plt.title('S_mean vs. Synergy Bliss', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/smean_v_bliss_ddinter.png')
plt.close()

# Plot scatter plot of synergy_bliss vs. S_sum
drug_syntox_df.plot.scatter(x='synergy_bliss', y='S_sum')
plt.xlabel('Synergy Bliss', fontsize=20)
plt.ylabel('S_sum', fontsize=20)
plt.title('S_sum vs. Synergy Bliss', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/ssum_v_bliss_ddinter.png')
plt.close()

# Plot scatter plot of synergy_hsa vs. synergy_loewe
drug_syntox_df.plot.scatter(x='synergy_hsa', y='synergy_loewe')
plt.xlabel('Synergy HSA', fontsize=20)
plt.ylabel('Synergy Loewe', fontsize=20)
plt.title('Synergy Loewe vs. Synergy HSA', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/loewe_v_hsa_ddinter.png')
plt.close()

# Plot scatter plot of synergy_hsa vs. synergy_zip
drug_syntox_df.plot.scatter(x='synergy_hsa', y='synergy_zip')
plt.xlabel('Synergy HSA', fontsize=20)
plt.ylabel('Synergy ZIP', fontsize=20)
plt.title('Synergy ZIP vs. Synergy HSA', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/zip_v_hsa_ddinter.png')
plt.close()

# Plot scatter plot of synergy_hsa vs. S_max
drug_syntox_df.plot.scatter(x='synergy_hsa', y='S_max')
plt.xlabel('Synergy HSA', fontsize=20)
plt.ylabel('S_max', fontsize=20)
plt.title('S_max vs. Synergy HSA', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/smax_v_hsa_ddinter.png')
plt.close()

# Plot scatter plot of synergy_hsa vs. S_mean
drug_syntox_df.plot.scatter(x='synergy_hsa', y='S_mean')
plt.xlabel('Synergy HSA', fontsize=20)
plt.ylabel('S_mean', fontsize=20)
plt.title('S_mean vs. Synergy HSA', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/smean_v_hsa_ddinter.png')
plt.close()

# Plot scatter plot of synergy_hsa vs. S_sum
drug_syntox_df.plot.scatter(x='synergy_hsa', y='S_sum')
plt.xlabel('Synergy HSA', fontsize=20)
plt.ylabel('S_sum', fontsize=20)
plt.title('S_sum vs. Synergy HSA', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/ssum_v_hsa_ddinter.png')
plt.close()

# Plot scatter plot of synergy_loewe vs. synergy_zip
drug_syntox_df.plot.scatter(x='synergy_loewe', y='synergy_zip')
plt.xlabel('Synergy Loewe', fontsize=20)
plt.ylabel('Synergy ZIP', fontsize=20)
plt.title('Synergy ZIP vs. Synergy Loewe', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/zip_v_loewe_ddinter.png')
plt.close()

# Plot scatter plot of synergy_loewe vs. S_max
drug_syntox_df.plot.scatter(x='synergy_loewe', y='S_max')
plt.xlabel('Synergy Loewe', fontsize=20)
plt.ylabel('S_max', fontsize=20)
plt.title('S_max vs. Synergy Loewe', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/smax_v_loewe_ddinter.png')
plt.close()

# Plot scatter plot of synergy_loewe vs. S_mean
drug_syntox_df.plot.scatter(x='synergy_loewe', y='S_mean')
plt.xlabel('Synergy Loewe', fontsize=20)
plt.ylabel('S_mean', fontsize=20)
plt.title('S_mean vs. Synergy Loewe', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/smean_v_loewe_ddinter.png')
plt.close()

# Plot scatter plot of synergy_loewe vs. S_sum
drug_syntox_df.plot.scatter(x='synergy_loewe', y='S_sum')
plt.xlabel('Synergy Loewe', fontsize=20)
plt.ylabel('S_sum', fontsize=20)
plt.title('S_sum vs. Synergy Loewe', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/ssum_v_loewe_ddinter.png')
plt.close()

# Plot scatter plot of synergy_zip vs. S_max
drug_syntox_df.plot.scatter(x='synergy_zip', y='S_max')
plt.xlabel('Synergy ZIP', fontsize=20)
plt.ylabel('S_max', fontsize=20)
plt.title('S_max vs. Synergy ZIP', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/smax_v_zip_ddinter.png')
plt.close()

# Plot scatter plot of synergy_zip vs. S_mean
drug_syntox_df.plot.scatter(x='synergy_zip', y='S_mean')
plt.xlabel('Synergy ZIP', fontsize=20)
plt.ylabel('S_mean', fontsize=20)
plt.title('S_mean vs. Synergy ZIP', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/smean_v_zip_ddinter.png')
plt.close()

# Plot scatter plot of synergy_zip vs. S_sum
drug_syntox_df.plot.scatter(x='synergy_zip', y='S_sum')
plt.xlabel('Synergy ZIP', fontsize=20)
plt.ylabel('S_sum', fontsize=20)
plt.title('S_sum vs. Synergy ZIP', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/ssum_v_zip_ddinter.png')
plt.close()

# Plot scatter plot of S_max vs. S_mean
drug_syntox_df.plot.scatter(x='S_max', y='S_mean')
plt.xlabel('S_max', fontsize=20)
plt.ylabel('S_mean', fontsize=20)
plt.title('S_mean vs. S_max', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/smean_v_smax_ddinter.png')
plt.close()

# Plot scatter plot of S_max vs. S_sum
drug_syntox_df.plot.scatter(x='S_max', y='S_sum')
plt.xlabel('S_max', fontsize=20)
plt.ylabel('S_sum', fontsize=20)
plt.title('S_sum vs. S_max', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/ssum_v_smax_ddinter.png')
plt.close()

# Plot scatter plot of S_mean vs. S_sum
drug_syntox_df.plot.scatter(x='S_mean', y='S_sum')
plt.xlabel('S_mean', fontsize=20)
plt.ylabel('S_sum', fontsize=20)
plt.title('S_sum vs. S_mean', fontsize=20)
plt.savefig('results/synergy_score_scatter_plots/ssum_v_smean_ddinter.png')
plt.close()

# for each pair of synergy scores, calculate the pearson correlation coefficient and write all to a file
synergy_scores = ['synergy_bliss', 'synergy_hsa', 'synergy_loewe', 'synergy_zip', 'S_max', 'S_mean', 'S_sum']
with open('results/synergy_score_scatter_plots/synergy_score_correlation_ddinter.tsv', 'w') as f:
    f.write('Synergy Score 1\tSynergy Score 2\tCorrelation Coefficient\tP-value\n')
    for score1, score2 in itertools.combinations(synergy_scores, 2):
        correlation = stats.pearsonr(drug_syntox_df[score1], drug_syntox_df[score2])
        f.write(f'{score1}\t{score2}\t{correlation[0]}\t{correlation[1]}\n')